In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|██████████| 26.4M/26.4M [00:02<00:00, 13.0MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 211kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.91MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 8.89MB/s]


In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
loss_fn = nn.CrossEntropyLoss()

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    # 예측과 손실 계산
    pred = model(X)
    loss = loss_fn(pred, y)

    # 역전파
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 ==0 :
      loss, current = loss.item(), batch*batch_size + len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
  # 모델을 평가 모드로 설정 - 배치 정규화 및 드롭아웃 레이어들에 중요
  model.eval()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0,0

  # torch.no_grad()를 사용해서 테스트 시 변화도를 꼐산하지 않도록 함
  # 이는 requires_grad = True로 설정된 텐서들의 불필요한 변화도 연산 및 메모리 사용량을 줄여줌

  with torch.no_grad():
    for X,y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

      test_loss /= num_batches
      correct /= size
      print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303972  [   64/60000]
loss: 2.294593  [ 6464/60000]
loss: 2.273967  [12864/60000]
loss: 2.269892  [19264/60000]
loss: 2.251688  [25664/60000]
loss: 2.220114  [32064/60000]
loss: 2.221190  [38464/60000]
loss: 2.189906  [44864/60000]
loss: 2.193305  [51264/60000]
loss: 2.148440  [57664/60000]
Test Error: 
 Accuracy: 0.4%, Avg loss: 0.013726 

Test Error: 
 Accuracy: 0.4%, Avg loss: 0.013664 

Test Error: 
 Accuracy: 0.4%, Avg loss: 0.013547 

Test Error: 
 Accuracy: 0.4%, Avg loss: 0.013688 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013799 

Test Error: 
 Accuracy: 0.4%, Avg loss: 0.013629 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013728 

Test Error: 
 Accuracy: 0.4%, Avg loss: 0.013749 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013864 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013773 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013800 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013840 

Test Error: 
 Accuracy: 0.3%, Avg loss: 0.013798